In [164]:
import numpy as np
import heapq

In [165]:
Distance = [[0,1064,673,1401,277],
            [1064,0,958,1934,377],
            [673,957,0,1001,399],
            [1401,1934,1001,0,387],
            [277,377,399,387,0]]
W_ = {1:{1:'A',2:'E',3:'D',4:'C',5:'A'},
        2:{1:'B',2:'E',3:'A',4:'C',5:'D'},
        3:{1:'B',2:'A',3:'B',4:'C',5:'E'},
        4:{1:'D',2:'A',3:'D',4:'B',5:'D'},
        5:{1:'B',2:'E',3:'C',4:'B',5:'D'}}
Starting_Point = ['A','B','C','D','E']
fac_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4}

In [166]:
mp ={}
for i in Starting_Point:
    dd = 20000
    for j in Starting_Point:
        if get_gscore_dist(i,j)<dd and i != j:
            dd = get_gscore_dist(i,j)
            mp[i]=j
print(mp)
for i in Starting_Point:
    for j in Starting_Point:
        if get_gscore_dist(i,j)>(get_gscore_dist(i,mp[i])+get_gscore_dist(mp[i],j)):
            Distance[fac_dict[i]][fac_dict[j]]=get_gscore_dist(i,mp[i])+get_gscore_dist(mp[i],j)
print(Distance)

{'A': 'E', 'B': 'E', 'C': 'E', 'D': 'E', 'E': 'A'}
[[0, 654, 673, 664, 277], [654, 0, 776, 764, 377], [673, 776, 0, 786, 399], [664, 764, 786, 0, 387], [277, 377, 399, 387, 0]]


In [7]:
Distance

[[0, 654, 673, 664, 277],
 [654, 0, 776, 764, 377],
 [673, 776, 0, 786, 399],
 [664, 764, 786, 0, 387],
 [277, 377, 399, 387, 0]]

MP2
===
You are the owner of a smart manufacturing company, with factories in five different cities: Atlanta, Boston, Chicago, Denver, and Edmonton. Each factory makes a different type of component: for convenience, let's call the components A, B, C, D, and E. The locations of your factories are a bit inconvenient to one another; the following table gives the distance (in miles) from one factory to another.

Distance | A | B | C | D | E
---------|---|---|---|---|---
A| 0 | 1064| 673 | 1401 | 277
B|1064| 0|	958|	1934|	337
C|673|958|0|	    1001|	    399
D|1401| 1934| 1001	|0|387
E|277|	337|	399|	387|	0
            
---
From those components, you manufacture five different types of widgets. Each widget is composed of five widgets of different types, as follows:

* Widget 1 = Components AEDCA
* Widget 2 = Components BEACD
* Widget 3 = Components BABCE
* Widget 4 = Components DADBD
* Widget 5 = Components BECBD
---
The goal of this assignment is to implement A* search that will find two different factory sequences:

* First, find the factory sequence with the smallest number of stops (the smallest number of factories visited).
* Second, find the factory sequence with the smallest number of miles traveled.

Solve this problem using A* search. Your report should specify your state representation, actions, and your heuristic. The heuristic need not be very smart, but it needs to be non-zero, and it needs to be admissible and consistent.

In [174]:
class node():
    def __init__(self, cur_factory,last_table, gscore=float('inf'),fscore=float('inf'), finished_factory=set(), parent=''):
        self.gscore = gscore
        self.last_table = last_table
        self.cur_factory = cur_factory
        self.parent = parent
        self.fscore = fscore
        self.finished_factory = finished_factory
    
    def __key(self):
        return (self.parent)
    
    def __eq__(self, other):
        return (np.array_equal(self.last_table,other.last_table) and self.gscore <= other.gscore)
    
    def __lt__(self, other):
        return self.fscore < other.fscore
    
    def __hash__(self):
        return hash(self.__key())
    

In [168]:
def h_min_dis(node,factory):
    table = node.last_table.copy()
    table2 = table.copy()
    table2 = table2.T
    fscore = 0
    for num, i in enumerate (table):
        if table2[num].sum() != 0:
            fscore=fscore+1000
        if i[0] == 1:
            if W_[num+1][1] == factory: i[0] = i[0] - 1
            continue
        elif i[1] == 1:
            if W_[num+1][2] == factory: i[1] = i[1] - 1 
            continue
        elif i[2] == 1:
            if W_[num+1][3] == factory: i[2] = i[2] - 1 
            continue
        elif i[3] == 1:
            if W_[num+1][4] == factory: i[3] = i[3] - 1
            continue    
        elif i[4] == 1:
            if W_[num+1][5] == factory: i[4] = i[4] - 1 
            continue
#     fscore = table.sum()
    
    return table,fscore
#     table = node.last_table.copy()
#     for num, i in enumerate (table):
#         if i[0] == 1:
#             if W_[num+1][1] == factory: i[0] = i[0] - 1
#             continue
#         elif i[1] == 1:
#             if W_[num+1][2] == factory: i[1] = i[1] - 1 
#             continue
#         elif i[2] == 1:
#             if W_[num+1][3] == factory: i[2] = i[2] - 1 
#             continue
#         elif i[3] == 1:
#             if W_[num+1][4] == factory: i[3] = i[3] - 1
#             continue    
#         elif i[4] == 1:
#             if W_[num+1][5] == factory: i[4] = i[4] - 1 
#             continue
#     fscore = Distance[fac_dict[node.cur_factory]][fac_dict[factory]]
#     return table,fscore                                      
    
def h_min_step(node,factory):
    '''
    calculate the huristic of a certain step of node
    '''
    table = node.last_table.copy()
    table2 = table.copy()
    table2 = table2.T
    fscore = 0
    for num, i in enumerate (table):
        if table2[num].sum() != 0:
            fscore=fscore+1
        if i[0] == 1:
            if W_[num+1][1] == factory: i[0] = i[0] - 1
            continue
        elif i[1] == 1:
            if W_[num+1][2] == factory: i[1] = i[1] - 1 
            continue
        elif i[2] == 1:
            if W_[num+1][3] == factory: i[2] = i[2] - 1 
            continue
        elif i[3] == 1:
            if W_[num+1][4] == factory: i[3] = i[3] - 1
            continue    
        elif i[4] == 1:
            if W_[num+1][5] == factory: i[4] = i[4] - 1 
            continue
#     fscore = table.sum()
    
    return table,fscore

def get_gscore_dist(city1,city2):
    return Distance[fac_dict[city1]][fac_dict[city2]]

def get_close(table):
    s = ''
    for i in table:
        j = i.sum()
        s = s+ str(int(j))
    return s

def get_low_rank(open_set):
    min_huris = float('inf')
    for nodes in open_set:
#         print(nodes.parent,nodes.fscore)
        if (nodes.fscore)<min_huris:
            cur_node = nodes
            min_huris = cur_node.fscore  
    return cur_node

def goal_test(node):
    if node.last_table.sum() == 0: return True
    return False

In [169]:
def A_star(start,hu):
    hu_table = np.ones((5,5))
    initnode = node(start,hu_table,gscore=0,fscore=25.0)
    table,fscore = hu(initnode,start)
    start_node = node(start,table,gscore=0,fscore=fscore,parent=start)
    close_set = {}
    open_set = set()
    open_set.add(start_node)
    cities = ['A','B','C','D','E']
    while len(open_set)!=0:
        min_huris = float('inf')
#         print('----starting choose lowest fscore----')
        for nodes in open_set:
#             print(nodes.parent,nodes.fscore)
            if (nodes.fscore)<min_huris:
                temp = nodes
                min_huris = temp.fscore
        cur_node = temp          
        if goal_test(cur_node): 
#             print(close_set)
            return cur_node.parent
        open_set.remove(cur_node)
#         print('choose=',cur_node.parent,' hur=',cur_node.fscore)
        for city in cities:
            if city == cur_node.cur_factory: continue
            table,fscore = hu(cur_node,city)
#             print('test fscore ',cur_node.parent+city,fscore+cur_node.gscore+1, 'cur fscore=',cur_node.fscore)
#             if fscore+cur_node.gscore+1 >= cur_node.fscore: continue
            if get_close(table) in close_set:
                if close_set[get_close(table)] < cur_node.gscore+1:
#                     print('useful')
                    continue
                close_set[get_close(table)] = cur_node.gscore+1
            new_node = node(city,table,gscore=cur_node.gscore+1,fscore=fscore+cur_node.gscore+1,parent=cur_node.parent+city)
#             print('----adding new node----')
#             print(new_node.parent,new_node.fscore)
#             print(new_node.last_table)
#             print()
            open_set.add(new_node)
            close_set[get_close(cur_node.last_table)]=cur_node.gscore
    print(close_set)
    return

In [ ]:
for t in Starting_Point:
    lenn = 1000
    print('for starting =',t)
    retv = A_star_d(t,h_min_dis,get_gscore_dist)
    dis = 0
    for s in retv:
        dis = dis + get_gscore_dist(t,s)
        t = s
    print(dis)
    print('temp=',retv)
    print('len=',len(retv))
    print()

In [9]:
W_

{1: {1: 'A', 2: 'E', 3: 'D', 4: 'C', 5: 'A'},
 2: {1: 'B', 2: 'E', 3: 'A', 4: 'C', 5: 'D'},
 3: {1: 'B', 2: 'A', 3: 'B', 4: 'C', 5: 'E'},
 4: {1: 'D', 2: 'A', 3: 'D', 4: 'B', 5: 'D'},
 5: {1: 'B', 2: 'E', 3: 'C', 4: 'B', 5: 'D'}}

In [ ]:
close_set = {}
hu_table = np.ones((5,5))
tnode = node('A',hu_table,gscore=0,fscore=25.0)
close_set[get_close(tnode)]=tnode.gscore

```
OPEN = priority queue containing START
CLOSED = empty set
while lowest rank in OPEN is not the GOAL:
  current = remove lowest rank item from OPEN
  add current to CLOSED
  for neighbors of current:
    cost = g(current) + movementcost(current, neighbor)
    if neighbor in OPEN and cost less than g(neighbor):
      remove neighbor from OPEN, because new path is better
    if neighbor in CLOSED and cost less than g(neighbor): ⁽²⁾
      remove neighbor from CLOSED
    if neighbor not in OPEN and neighbor not in CLOSED:
      set g(neighbor) to cost
      add neighbor to OPEN
      set priority queue rank to g(neighbor) + h(neighbor)
      set neighbor's parent to current
```

In [190]:
def A_star_d(start,hu,ge):
    RESULT=[]
    hu_table = np.ones((5,5))
    initnode = node(start,hu_table,gscore=0,fscore=25.0)
    table,fscore = hu(initnode,start)
    start_node = node(start,table,gscore=0,fscore=fscore,parent=start)
    close_set = {}
    open_set = []
    heapq.heappush(open_set, start_node)
#     open_set.add(start_node)
    while len(open_set)!=0:
#         print(len(open_set))
        cur_node = heapq.heappop(open_set)
        if goal_test(cur_node): 
#             print(close_set)
#             return cur_node.parent
            RESULT.append(cur_node.parent)
        close_set[get_close(cur_node.last_table)]=cur_node.gscore
#         print('choose=',cur_node.parent,' hur=',cur_node.fscore)
        for city in Starting_Point:
            if city == cur_node.cur_factory: continue
            table,fscore = hu(cur_node,city)
            cost = cur_node.gscore+ge(cur_node.cur_factory,city)
            new_node = node(city,table,gscore=cost,fscore=fscore+cost,parent=cur_node.parent+city)
#             print('test fscore ',cur_node.parent+city,fscore+cur_node.gscore+ge(cur_node.cur_factory,city), 'cur fscore=',cur_node.fscore)
#             if fscore+cur_node.gscore+1 >= cur_node.fscore: continue
#             print(get_close(table))
#             print(close_set)
            if get_close(table) in close_set:
#                 print(new_node.parent,' close_set',get_close(table),'=',close_set[get_close(table)],' cur=',cost)
                if close_set[get_close(table)] < cost: continue
                else:
                    print('im else!!!!!!!!--------------##########')
                    close_set[get_close(table)] = cost
            if(new_node not in open_set):
                heapq.heappush(open_set,new_node)
#                 print('----adding new node----')
#                 print(new_node.parent,new_node.fscore)
#                 print(new_node.last_table)
#                 print()
#                 print(close_set)
#     print(close_set)
    return RESULT

In [191]:
sss = A_star_d('A',h_min_dis,get_gscore_dist)

im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########
im else!!!!!!!!--------------##########


In [123]:
ss

'AEBEDCADBCED'

In [192]:
sss

['ABEDADBCBEAD', 'ABEDADBCBEDA', 'ABEDADBCEABD']

In [193]:
Distance

[[0, 654, 673, 664, 277],
 [654, 0, 776, 764, 377],
 [673, 776, 0, 786, 399],
 [664, 764, 786, 0, 387],
 [277, 377, 399, 387, 0]]

In [194]:
for i in sss:
    startttt = 'A'
    dis = 0
    for s in i:
        dis = dis + get_gscore_dist(startttt,s)
        startttt = s
    print(i,dis)
    

ABEDADBCBEAD 6380
ABEDADBCBEDA 6490
ABEDADBCEABD 6380


In [22]:
W_

{1: {1: 'A', 2: 'E', 3: 'D', 4: 'C', 5: 'A'},
 2: {1: 'B', 2: 'E', 3: 'A', 4: 'C', 5: 'D'},
 3: {1: 'B', 2: 'A', 3: 'B', 4: 'C', 5: 'E'},
 4: {1: 'D', 2: 'A', 3: 'D', 4: 'B', 5: 'D'},
 5: {1: 'B', 2: 'E', 3: 'C', 4: 'B', 5: 'D'}}

In [ ]:
hu_table = np.ones((5,5))
initnode = node('A',hu_table,gscore=0,fscore=25.0)
table,fscore = h_min_dis(initnode,'A')
start_node = node('A',table,gscore=0,fscore=fscore,parent='A')
close_set = {}
open_set = set()
open_set.add(start_node)
cities = ['A','B','C','D','E']
while len(open_set)!=0:
    min_huris = float('inf')
#         print('----starting choose lowest fscore----')
    for nodes in open_set:
#             print(nodes.parent,nodes.fscore)
        if (nodes.fscore)<min_huris:
            temp = nodes
            min_huris = temp.fscore
    cur_node = temp          
    if goal_test(cur_node): 
#             print(close_set)
        print(cur_node.parent)
    open_set.remove(cur_node)
#         print('choose=',cur_node.parent,' hur=',cur_node.fscore)
    for city in cities:
        if city == cur_node.cur_factory: continue
        table,fscore = h_min_dis(cur_node,city)
#             print('test fscore ',cur_node.parent+city,fscore+cur_node.gscore+ge(cur_node.cur_factory,city), 'cur fscore=',cur_node.fscore)
#             if fscore+cur_node.gscore+1 >= cur_node.fscore: continue
        if get_close(table) in close_set:
                if close_set[get_close(table)] < cur_node.gscore+ge(cur_node.cur_factory,city):
#                     print('useful')
                    continue
            close_set[get_close(table)] = cur_node.gscore+get_gscore_dist(cur_node.cur_factory,city)
        new_node = node(city,table,gscore=cur_node.gscore+get_gscore_dist(cur_node.cur_factory,city),fscore=fscore+cur_node.gscore+get_gscore_dist(cur_node.cur_factory,city),parent=cur_node.parent+city)
            print('----adding new node----')
            print(new_node.parent,new_node.fscore)
            print(new_node.last_table)
            print()
        open_set.add(new_node)
        close_set[get_close(cur_node.last_table)]=cur_node.gscore
#     print(close_set)

In [ ]:
close_set

In [112]:
o = set()
hu_table = np.ones((5,5))
initnode = node('A',hu_table,gscore=0,fscore=25.0)
o.add(initnode)
sssec = node('A',hu_table,gscore=0,fscore=25.0)

In [113]:
sssec in o

True

In [61]:
initnode

In [105]:
np.array_equal(np.ones((5,5)),np.ones((5,5)))

True